# This Script is implemented for Parallel Processing

## Importing Libraries 📚

In [3]:
import time
import os
from pandarallel import pandarallel
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import word_tokenize
from nltk import ngrams

In [4]:
Beginning = time.time()
n_workers = max(1, os.cpu_count() // 2)
pandarallel.initialize(nb_workers=n_workers, progress_bar=True)
print(f"\nUsing {n_workers} cores for parallel processing")

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/

Using 10 cores for parallel processing


In [5]:
start = time.time()
nltk.download('punkt_tab')
nltk.download('stopwords')

end = time.time()
times = []
times.append(["Download", end-start])
print("✅ Finished")

✅ Finished


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Saad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Studying and Processing Data 🔍 & 🛠️

In [7]:
pd.set_option('display.max_colwidth', 100)
data = pd.read_csv('arabic_sentiment_reviews.csv', encoding='utf-8')

print("Data loaded successfully. Number of reviews:", len(data))
print("Class distribution:\n", data['label'].value_counts())

Data loaded successfully. Number of reviews: 330000
Class distribution:
 label
1    166853
0    163147
Name: count, dtype: int64


In [8]:
data.head()

,label,content
0,1,النعال المريحة: أرتدي هذه النعال كثيرًا!فهي دافئة ومريحة وبأسعار معقولة لجودة رائعة.زوجي وأنا عل...
1,1,منتج جميل ، خدمة سيئة: لقد اشتريت زوجًا من النعال الباو الدب.باتباع إرشادات وصف المنتج ، ارتفعت ...
2,1,جيد للأشياء الصغيرة: هذا يعمل بشكل جيد لالتقاط قطع صغيرة من المجوهرات ، ولكن الذهاب ببطء.إنه مفي...
3,0,واهية للغاية: flimsyif للغاية ، فأنت تشتريه ، كن حذرًا جدًا للغاية مع إطالة وتقصيره ، وأي استخدا...
4,1,Pop for Girls and Girly Boys ، والأشخاص الذين يحبون الضحك: عليك فقط أن تبتسم عندما تستمع إلى Boo...


In [9]:
def preprocess_text(text):
    import re
    # Compile patterns inside function for multiprocessing safety
    non_arabic_pattern = re.compile(r'[^\u0621-\u064A\s]')
    alif_pattern = re.compile(r'[أإآ]')
    ya_pattern = re.compile(r'ى')
    ta_marbuta_pattern = re.compile(r'ة')
    diacritics_pattern = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    tatweel_pattern = re.compile(r'ـ')
    
    # Apply replacements
    text = diacritics_pattern.sub('', text)
    text = tatweel_pattern.sub('', text)
    text = non_arabic_pattern.sub(' ', text)
    text = alif_pattern.sub('ا', text)
    text = ya_pattern.sub('ي', text)
    text = ta_marbuta_pattern.sub('ه', text)
    
    return text.strip()

print("\nCleaning and normalizing text...")
start = time.time()
data['cleaned_content'] = data['content'].parallel_apply(preprocess_text)
end = time.time()
times.append(["Normalizing", end-start])
print("✅ Finished")


Cleaning and normalizing text...


✅ Finished


In [10]:
print("Tokenizing and removing stop words...")
arabic_stopwords = set(stopwords.words('arabic'))

def remove_stopwords(tokens, stopwords_set=arabic_stopwords):
    # Import inside function is safer on Windows
    return [word for word in tokens if word not in stopwords_set]
    
start = time.time()
data['tokenized_content'] = data['cleaned_content'].parallel_apply(word_tokenize)
data['filtered_content'] = data['tokenized_content'].parallel_apply(remove_stopwords)
end = time.time()
times.append(["Tokenizing and removing stop words", end-start])
print("✅ Finished")

Tokenizing and removing stop words...


✅ Finished


In [11]:
print("Stemming...")
stemmer = ISRIStemmer()
def stem_tokens(tokens, stemmer_instance=stemmer):
    return [stemmer_instance.stem(word) for word in tokens]
    
start = time.time()
stemmer = ISRIStemmer()
data['stemmed_content'] = data['filtered_content'].parallel_apply(stem_tokens)
end = time.time()
times.append(["Stemming", end-start])
print("✅ Finished")

Stemming...


✅ Finished


In [12]:
def generate_2grams(tokens):
    from nltk import ngrams
    return ['_'.join(gram) for gram in ngrams(tokens, 2)]

def generate_3grams(tokens):
    from nltk import ngrams
    return ['_'.join(gram) for gram in ngrams(tokens, 3)]

def generate_4grams(tokens):
    from nltk import ngrams
    return ['_'.join(gram) for gram in ngrams(tokens, 4)]

print("\n Generating n-grams (2, 3, 4)...")
start = time.time()
data['bigrams'] = data['stemmed_content'].parallel_apply(generate_2grams)
data['trigrams'] = data['stemmed_content'].parallel_apply(generate_3grams)
data['fourgrams'] = data['stemmed_content'].parallel_apply(generate_4grams)
end = time.time()
times.append(["N-gram generation", end-start])
print("✅ Finished")


 Generating n-grams (2, 3, 4)...


✅ Finished


In [13]:
data.head()

,label,content,cleaned_content,tokenized_content,filtered_content,stemmed_content,bigrams,trigrams,fourgrams
0,1,النعال المريحة: أرتدي هذه النعال كثيرًا!فهي دافئة ومريحة وبأسعار معقولة لجودة رائعة.زوجي وأنا عل...,النعال المريحه ارتدي هذه النعال كثيرا فهي دافئه ومريحه وباسعار معقوله لجوده رائعه زوجي وانا علي...,"[النعال, المريحه, ارتدي, هذه, النعال, كثيرا, فهي, دافئه, ومريحه, وباسعار, معقوله, لجوده, رائعه, ...","[النعال, المريحه, ارتدي, النعال, فهي, دافئه, ومريحه, وباسعار, معقوله, لجوده, رائعه, زوجي, وانا, ...","[نعل, ريح, ردي, نعل, فهي, دفئ, ريح, سعر, عقل, لجد, رئع, زوج, ونا, علي, حد, سوء, لدي, زوج, ونح, نحب]","[نعل_ريح, ريح_ردي, ردي_نعل, نعل_فهي, فهي_دفئ, دفئ_ريح, ريح_سعر, سعر_عقل, عقل_لجد, لجد_رئع, رئع_ز...","[نعل_ريح_ردي, ريح_ردي_نعل, ردي_نعل_فهي, نعل_فهي_دفئ, فهي_دفئ_ريح, دفئ_ريح_سعر, ريح_سعر_عقل, سعر_...","[نعل_ريح_ردي_نعل, ريح_ردي_نعل_فهي, ردي_نعل_فهي_دفئ, نعل_فهي_دفئ_ريح, فهي_دفئ_ريح_سعر, دفئ_ريح_سع..."
1,1,منتج جميل ، خدمة سيئة: لقد اشتريت زوجًا من النعال الباو الدب.باتباع إرشادات وصف المنتج ، ارتفعت ...,منتج جميل خدمه سيئه لقد اشتريت زوجا من النعال الباو الدب باتباع ارشادات وصف المنتج ارتفعت ح...,"[منتج, جميل, خدمه, سيئه, لقد, اشتريت, زوجا, من, النعال, الباو, الدب, باتباع, ارشادات, وصف, المنت...","[منتج, جميل, خدمه, سيئه, لقد, اشتريت, زوجا, النعال, الباو, الدب, باتباع, ارشادات, وصف, المنتج, ا...","[نتج, جمل, خدم, سيئ, لقد, شري, زوج, نعل, باو, لدب, تبع, رشد, وصف, نتج, رفع, حجم, نسب, عند, وصل, ...","[نتج_جمل, جمل_خدم, خدم_سيئ, سيئ_لقد, لقد_شري, شري_زوج, زوج_نعل, نعل_باو, باو_لدب, لدب_تبع, تبع_ر...","[نتج_جمل_خدم, جمل_خدم_سيئ, خدم_سيئ_لقد, سيئ_لقد_شري, لقد_شري_زوج, شري_زوج_نعل, زوج_نعل_باو, نعل_...","[نتج_جمل_خدم_سيئ, جمل_خدم_سيئ_لقد, خدم_سيئ_لقد_شري, سيئ_لقد_شري_زوج, لقد_شري_زوج_نعل, شري_زوج_نع..."
2,1,جيد للأشياء الصغيرة: هذا يعمل بشكل جيد لالتقاط قطع صغيرة من المجوهرات ، ولكن الذهاب ببطء.إنه مفي...,جيد للاشياء الصغيره هذا يعمل بشكل جيد لالتقاط قطع صغيره من المجوهرات ولكن الذهاب ببطء انه مفي...,"[جيد, للاشياء, الصغيره, هذا, يعمل, بشكل, جيد, لالتقاط, قطع, صغيره, من, المجوهرات, ولكن, الذهاب, ...","[جيد, للاشياء, الصغيره, يعمل, بشكل, جيد, لالتقاط, قطع, صغيره, المجوهرات, الذهاب, ببطء, انه, مفيد...","[جيد, شيء, صغر, عمل, شكل, جيد, لقط, قطع, صغر, جوهر, ذهب, بطء, انه, فيد, مغناطيس, قوا, جدا]","[جيد_شيء, شيء_صغر, صغر_عمل, عمل_شكل, شكل_جيد, جيد_لقط, لقط_قطع, قطع_صغر, صغر_جوهر, جوهر_ذهب, ذهب...","[جيد_شيء_صغر, شيء_صغر_عمل, صغر_عمل_شكل, عمل_شكل_جيد, شكل_جيد_لقط, جيد_لقط_قطع, لقط_قطع_صغر, قطع_...","[جيد_شيء_صغر_عمل, شيء_صغر_عمل_شكل, صغر_عمل_شكل_جيد, عمل_شكل_جيد_لقط, شكل_جيد_لقط_قطع, جيد_لقط_قط..."
3,0,واهية للغاية: flimsyif للغاية ، فأنت تشتريه ، كن حذرًا جدًا للغاية مع إطالة وتقصيره ، وأي استخدا...,واهيه للغايه للغايه فانت تشتريه كن حذرا جدا للغايه مع اطاله وتقصيره واي استخدام ...,"[واهيه, للغايه, للغايه, فانت, تشتريه, كن, حذرا, جدا, للغايه, مع, اطاله, وتقصيره, واي, استخدام, ع...","[واهيه, للغايه, للغايه, فانت, تشتريه, حذرا, جدا, للغايه, اطاله, وتقصيره, واي, استخدام, علي, الاط...","[وهي, غيه, غيه, فنت, شري, حذر, جدا, غيه, طله, قصر, واي, خدم, علي, طلق, علي, طلق, بلا, زاح]","[وهي_غيه, غيه_غيه, غيه_فنت, فنت_شري, شري_حذر, حذر_جدا, جدا_غيه, غيه_طله, طله_قصر, قصر_واي, واي_خ...","[وهي_غيه_غيه, غيه_غيه_فنت, غيه_فنت_شري, فنت_شري_حذر, شري_حذر_جدا, حذر_جدا_غيه, جدا_غيه_طله, غيه_...","[وهي_غيه_غيه_فنت, غيه_غيه_فنت_شري, غيه_فنت_شري_حذر, فنت_شري_حذر_جدا, شري_حذر_جدا_غيه, حذر_جدا_غي..."
4,1,Pop for Girls and Girly Boys ، والأشخاص الذين يحبون الضحك: عليك فقط أن تبتسم عندما تستمع إلى Boo...,والاشخاص الذين يحبون الضحك عليك فقط ان تبتسم عندما تستمع الي وهم فرقه كلها هناك ...,"[والاشخاص, الذين, يحبون, الضحك, عليك, فقط, ان, تبتسم, عندما, تستمع, الي, وهم, فرقه, كلها, هناك, ...","[والاشخاص, يحبون, الضحك, فقط, ان, تبتسم, عندما, تستمع, الي, وهم, فرقه, كلها, وهم, ممتعون, عندما,...","[شخص, يحب, ضحك, فقط, ان, بسم, عند, سمع, الي, وهم, فرق, كله, وهم, متع, عند, تخذ, الي, جده, ان, تك...","[شخص_يحب, يحب_ضحك, ضحك_فقط, فقط_ان, ان_بسم, بسم_عند, عند_سمع, سمع_الي, الي_وهم, وهم_فرق, فرق_كله...","[شخص_يحب_ضحك, يحب_ضحك_فقط, ضحك_فقط_ان, فقط_ان_بسم, ان_بسم_عند, بسم_عند_سم

In [14]:
Final = time.time()
times.append(["Overall", Final - Beginning])

print("Times for Parallel Processing")
for i in times:
    print(f"⏱️ {i[0]} : {i[1]:.4} seconds")

Times for Parallel Processing
⏱️ Download : 1.012 seconds
⏱️ Normalizing : 3.383 seconds
⏱️ Tokenizing and removing stop words : 23.19 seconds
⏱️ Stemming : 15.54 seconds
⏱️ N-gram generation : 46.49 seconds
⏱️ Overall : 91.84 seconds
